In [1]:
import logging
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss
import os
from os.path import join
import sys
import pickle
import csv
from transformers import BertTokenizer,BertForSequenceClassification
from transformers.optimization import AdamW
from tqdm import tqdm_notebook, trange
from multiprocessing import Pool, cpu_count

I0127 15:39:56.776237 140644016244480 file_utils.py:35] PyTorch version 1.4.0 available.


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(6)

In [21]:
# set dimension of interest
dim = 'romance'

In [3]:
DATA_DIR = 'data/%s'%dim # directory of where the train/test/dev files are stored
OUTPUT_DIR = 'weights/BERT/%s' %dim # where the model weights will be stored
BERT_MODEL = 'bert-base-cased' # BERT model type
CACHE_DIR = 'cache/' # where BERT will look for pre-trained models to load parameters from

num_labels = 2

OUTPUT_MODE = 'classification'
CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [4]:
# load model
model = BertForSequenceClassification.from_pretrained(BERT_MODEL,cache_dir=CACHE_DIR, num_labels=num_labels)

I0127 15:40:14.703091 140644016244480 configuration_utils.py:185] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at cache/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.d7a3af18ce3a2ab7c0f48f04dc8daff45ed9a3ed333b9e9a79d012a0dedf87a6
I0127 15:40:14.707180 140644016244480 configuration_utils.py:199] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscrip

In [7]:
# load pretrained BERT model for specific dimension
output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
model.load_state_dict(torch.load(output_model_file))
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [10]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False)

I0127 15:41:21.850724 140644016244480 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /home/minje/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [18]:
## enter a sentence
rand_sentence = '### insert sentence here ###'
rand_sentence = 'this man is crazy in love'

In [25]:
input_ids = torch.tensor([tokenizer.encode(rand_sentence,add_special_tokens=True)]).cuda()
with torch.no_grad():
    outputs = model(input_ids)[0]
    score = torch.softmax(outputs,1)
print("%s score for sentence: %1.3f"%(dim,score[0,1].item()))

romance score for sentence: 0.229
